In [4]:
import subprocess

subprocess.check_call(["pip", "install", "speechrecognition", "pyttsx3", "openai", "python-dotenv"])

0

In [10]:
!pip install speechrecognition pyttsx3 openai python-dotenv
!pip install pyaudio 
!pip install SpeechRecognition pyttsx3 transformers torch Flask



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import speech_recognition as sr

def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio)
            print("You said: " + text)
            return text
        except sr.UnknownValueError:
            print("Sorry, I could not understand the audio.")
            return None
        except sr.RequestError:
            print("Could not request results from Google Speech Recognition service.")
            return None

In [12]:
from transformers import pipeline

def generate_response(input_text):
    generator = pipeline('text-generation', model='gpt2')  # Replace 'gpt2' with your chosen model
    response = generator(input_text, max_length=50, num_return_sequences=1)
    return response[0]['generated_text']

In [14]:
import pyttsx3

def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

In [15]:
from flask import Flask, render_template, request
import threading

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/ask', methods=['POST'])
def ask():
    user_input = request.form['user_input']
    response = generate_response(user_input)
    speak(response)
    return response

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\HIMAJA\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [16]:
from flask import Flask, render_template, request
import speech_recognition as sr
import pyttsx3
from transformers import pipeline

app = Flask(__name__)

# Initialize the text-to-speech engine
engine = pyttsx3.init()

# Initialize the LLM
generator = pipeline('text-generation', model='gpt2')  # You can replace 'gpt2' with your chosen model

def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio)
            print("You said: " + text)
            return text
        except sr.UnknownValueError:
            print("Sorry, I could not understand the audio.")
            return None
        except sr.RequestError:
            print("Could not request results from Google Speech Recognition service.")
            return None

def generate_response(input_text):
    response = generator(input_text, max_length=50, num_return_sequences=1)
    return response[0]['generated_text']

def speak(text):
    engine.say(text)
    engine.runAndWait()

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/ask', methods=['POST'])
def ask():
    user_input = request.form['user_input']
    response = generate_response(user_input)
    speak(response)
    return response

if __name__ == '__main__':
    app.run(debug=True)

c:\Users\HIMAJA\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HIMAJA\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

In [9]:
import speech_recognition as sr
import pyttsx3
import openai
import time
import os
from dotenv import load_dotenv

# Load API key from environment variable (for OpenAI GPT usage)
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Initialize Text-to-Speech engine
engine = pyttsx3.init()

def speech_to_text():
    """Capture speech input and convert to text."""
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        try:
            audio = recognizer.listen(source, timeout=5)
            print("Processing...")
            text = recognizer.recognize_google(audio)
            return text
        except sr.WaitTimeoutError:
            print("Listening timed out.")
            return None
        except sr.UnknownValueError:
            print("Sorry, I could not understand the speech.")
            return None

def generate_response(input_text):
    """Generate a response using OpenAI GPT-3 or a similar LLM."""
    try:
        start_time = time.time()  # Measure performance
        response = openai.Completion.create(
            engine="text-davinci-003",  # Use GPT-3 engine
            prompt=input_text,
            max_tokens=100
        )
        end_time = time.time()
        print(f"Response generated in {round(end_time - start_time, 2)} seconds")
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print("Error generating response:", e)
        return None

def text_to_speech(text):
    """Convert text to speech."""
    engine.say(text)
    engine.runAndWait()

def main():
    print("Welcome to the Speech-to-Speech LLM Bot!")
    while True:
        # Step 1: Capture speech input
        user_input = speech_to_text()
        if user_input is None:
            print("No input detected. Try again.")
            continue
        print(f"You said: {user_input}")

        # Step 2: Generate LLM response
        print("Generating response...")
        llm_response = generate_response(user_input)
        if llm_response is None:
            print("Failed to generate a response.")
            continue
        print(f"Bot Response: {llm_response}")

        # Step 3: Convert LLM response to speech
        print("Speaking response...")
        text_to_speech(llm_response)

        # Exit condition
        if "exit" in user_input.lower():
            print("Exiting the bot. Goodbye!")
            break

if __name__ == "__main__":
    main()


Welcome to the Speech-to-Speech LLM Bot!
Listening...
Processing...
Sorry, I could not understand the speech.
No input detected. Try again.
Listening...
Processing...
Sorry, I could not understand the speech.
No input detected. Try again.
Listening...
Processing...
Sorry, I could not understand the speech.
No input detected. Try again.
Listening...
Processing...
Sorry, I could not understand the speech.
No input detected. Try again.
Listening...
Processing...
Sorry, I could not understand the speech.
No input detected. Try again.
Listening...
Processing...
Sorry, I could not understand the speech.
No input detected. Try again.
Listening...
Processing...
Sorry, I could not understand the speech.
No input detected. Try again.
Listening...
Processing...
Sorry, I could not understand the speech.
No input detected. Try again.
Listening...
Processing...
Sorry, I could not understand the speech.
No input detected. Try again.
Listening...
Processing...
Sorry, I could not understand the speech.

KeyboardInterrupt: 